# Project: [Analyze International Debt Statistics](https://projects.datacamp.com/projects/1906)

It's not that we humans only take debts to manage our necessities. A country may also take debt to manage its economy. For example, infrastructure spending is one costly ingredient required for a country's citizens to lead comfortable lives. [The World Bank](https://www.worldbank.org) is the organization that provides debt to countries.

In this notebook, we are going to analyze international debt data collected by The World Bank. The dataset contains information about the amount of debt (in USD) owed by **developing countries** across several categories. We are going to find the answers to questions like: 

- What is the total amount of debt that is owed by the countries listed in the dataset?
- Which country owns the maximum amount of debt and what does that amount look like?
- What is the average amount of debt owed by countries across different debt indicators?

![](image.jpg)

Below is a snapshot of the database you will be working with:

|country_name|country_code|indicator_name                                                    |indicator_code|debt       |
|------------|------------|------------------------------------------------------------------|--------------|-----------|
|Afghanistan |AFG         |"Disbursements on external debt, long-term (DIS, current US$)"    |DT.DIS.DLXF.CD|72894453.7 |
|Afghanistan |AFG         |"Interest payments on external debt, long-term (INT, current US$)"|DT.INT.DLXF.CD|53239440.1 |
|Afghanistan |AFG         |"PPG, bilateral (AMT, current US$)"                               |DT.AMT.BLAT.CD|61739336.9 |
|Afghanistan |AFG         |"PPG, bilateral (DIS, current US$)"                               |DT.DIS.BLAT.CD|49114729.4 |
|Afghanistan |AFG         |"PPG, bilateral (INT, current US$)"                               |DT.INT.BLAT.CD|39903620.1 |
|Afghanistan |AFG         |"PPG, multilateral (AMT, current US$)"                            |DT.AMT.MLAT.CD|39107845   |
|Afghanistan |AFG         |"PPG, multilateral (DIS, current US$)"                            |DT.DIS.MLAT.CD|23779724.3 |
|Afghanistan |AFG         |"PPG, multilateral (INT, current US$)"                            |DT.INT.MLAT.CD|13335820   |
|Afghanistan |AFG         |"PPG, official creditors (AMT, current US$)"                      |DT.AMT.OFFT.CD|100847181.9|
|Afghanistan |AFG         |"PPG, official creditors (DIS, current US$)"                      |DT.DIS.OFFT.CD|72894453.7 |

You will execute SQL queries to answer six questions, as listed in the instructions.

In [1]:
%load_ext sql
%sql duckdb:// --alias international_debt
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False

Connecting to 'international_debt'

In [2]:
%sql DROP TABLE IF EXISTS international_debt;
%sql CREATE TABLE international_debt AS SELECT * FROM read_csv_auto('data/international_debt.csv', types={'country_name': 'VARCHAR', 'country_code': 'VARCHAR','indicator_name': 'TEXT', 'indicator_code': 'TEXT', 'debt': 'NUMERIC'});
%sql SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'international_debt'

,column_name,data_type
0,country_name,VARCHAR
1,country_code,VARCHAR
2,indicator_name,VARCHAR
3,indicator_code,VARCHAR
4,debt,"DECIMAL(18,3)"


1. What is the number of distinct countries present in the database? The output should be single column aliased with the following name: num_distinct_countries.

In [3]:
%%sql international_debt
--num_distinct_countries
SELECT COUNT(DISTINCT international_debt.country_name) AS total_distinct_countries
FROM international_debt;

,total_distinct_countries
0,124


2. What are the distinct debt indicators? the output column should be aliased as distinct_debt_indicators and the outputs should be ordered by it.
What is the total amount of debt owed by all the countries present in the table, in millions? The output should be single column aliased with the following name: total_debt.

In [4]:
%%sql international_debt
--distinct_debt_indicators
SELECT DISTINCT international_debt.indicator_code AS distinct_debt_indicators
FROM international_debt
ORDER BY distinct_debt_indicators ASC;

,distinct_debt_indicators
0,DT.AMT.BLAT.CD
1,DT.AMT.DLXF.CD
2,DT.AMT.DPNG.CD
3,DT.AMT.MLAT.CD
4,DT.AMT.OFFT.CD
5,DT.AMT.PBND.CD
6,DT.AMT.PCBK.CD
7,DT.AMT.PROP.CD
8,DT.AMT.PRVT.CD
9,DT.DIS.BLAT.CD


3. What is the total amount of debt owed by all the countries present in the table, in millions? The output should be single column aliased with the following name: total_debt.

In [5]:
%%sql international_debt
--total_debt
SELECT ROUND(SUM(international_debt.debt)/1000000,2) AS total_debt
FROM international_debt

,total_debt
0,3079734.49


4. What country has the highest amount of debt?
   - Select the country_name and debt columns, then apply the SUM function on the debt column.
   - Alias the column resulted from the summation as total_debt.
   - Order BY total_debt descending
   - LIMIT to one row

In [6]:
%%sql international_debt
--highest_debt_country
SELECT
	international_debt.country_name AS country_name,	
	SUM(international_debt.debt) AS total_debt
FROM international_debt
GROUP BY international_debt.country_name
ORDER BY total_debt DESC
LIMIT 1;

,country_name,total_debt
0,China,2.857935e+11


5. What is the average amount of debt across different debt indicators?
   - Select indicator_code aliased as debt_indicator, then select indicator_name and debt.
   - Sort the output with respect to the average_debt column in a descending manner and limit the results to ten.

In [7]:
%%sql international_debt
--avg_debt_per_indicator
SELECT
	international_debt.indicator_code AS debt_indicator,
	international_debt.indicator_name,
	AVG(international_debt.debt) AS average_debt
FROM international_debt
GROUP BY debt_indicator, international_debt.indicator_name
ORDER BY average_debt DESC
LIMIT 10;


,debt_indicator,indicator_name,average_debt
0,DT.AMT.DLXF.CD,"""Principal repayments on external debt, long-t...",5.904868e+09
1,DT.AMT.DPNG.CD,"""Principal repayments on external debt, privat...",5.161194e+09
2,DT.DIS.DLXF.CD,"""Disbursements on external debt, long-term (DI...",2.152041e+09
3,DT.DIS.OFFT.CD,"""PPG, official creditors (DIS, current US$)""",1.958983e+09
4,DT.AMT.PRVT.CD,"""PPG, private creditors (AMT, current US$)""",1.803694e+09
5,DT.INT.DLXF.CD,"""Interest payments on external debt, long-term...",1.644024e+09
6,DT.DIS.BLAT.CD,"""PPG, bilateral (DIS, current US$)""",1.223139e+09
7,DT.INT.DPNG.CD,"""Interest payments on external debt, private n...",1.220411e+09
8,DT.AMT.OFFT.CD,"""PPG, official creditors (AMT, current US$)""",1.191188e+09
9,DT.AMT.PBND.CD,"""PPG, bonds (AMT, current US$)""",1.082624e+09


6. What is the highest amount of principal repayments in the "DT.AMT.DLXF.CD" category?

   - Select the country_name and indicator_name columns.


In [9]:
%%sql international_debt
--highest_principal_repayment
SELECT 
	international_debt.country_name,
	international_debt.indicator_name
FROM international_debt
WHERE international_debt.indicator_code = 'DT.AMT.DLXF.CD'
ORDER BY international_debt.debt DESC
LIMIT 1;

,country_name,indicator_name
0,China,"""Principal repayments on external debt, long-t..."
